# Sentencepiece를 활용해 Vocab 만들기

## 1. 말뭉치 만들기(한국어위키)

### [web-crawler](https://github.com/paul-hyun/web-crawler.git) 를 이용해서 말뭉치 다운로드 받고 변환


In [1]:
import pandas as pd

in_file = '../../data/kowiki/kowiki_20200722.csv'
out_file = '../../data/kowiki/kowiki.txt'
SEPARATOR = u"\u241D"
df = pd.read_csv(in_file, sep=SEPARATOR, engine="python")
with open(out_file, "w") as f:
    for index, row in df.iterrows():
        f.write(row["text"]) # title 과 text를 중복 되므로 text만 저장 함
        f.write("\n\n\n\n") # 구분자

## 2. Vocab 만들기

In [11]:
import sentencepiece as spm

corpus = '../../data/kowiki/kowiki_25_percent.txt'
prefix = 'kowiki'
vocab_size = 8000

spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe" +
    " --max_sentence_length=999999" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]") # 사용자 정의 토큰


## 3. Vocab 테스트

In [12]:
import sentencepiece as spm

vocab_file = './kowiki.model'
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

lines = [
  "겨울이 되어서 날씨가 무척 추워요.",
  "이번 성탄절은 화이트 크리스마스가 될까요?",
  "겨울에 감기 조심하시고 행복한 연말 되세요."
]

for line in lines:
    pieces = vocab.encode_as_pieces(line)
    ids = vocab.encode_as_ids(line)
    print(line)
    print(pieces)
    print(ids)
    print()


겨울이 되어서 날씨가 무척 추워요.
['▁겨울', '이', '▁되어', '서', '▁날', '씨', '가', '▁무', '척', '▁추', '워', '요', '.']
[2605, 3774, 547, 3789, 638, 4210, 3784, 103, 4363, 189, 4150, 3939, 3778]

이번 성탄절은 화이트 크리스마스가 될까요?
['▁이', '번', '▁성', '탄', '절', '은', '▁화', '이트', '▁크리스', '마', '스가', '▁될', '까', '요', '?']
[7, 4002, 78, 4126, 4095, 3790, 260, 786, 2730, 3858, 703, 1347, 3980, 3939, 4488]

겨울에 감기 조심하시고 행복한 연말 되세요.
['▁겨울', '에', '▁감', '기', '▁조', '심', '하', '시', '고', '▁행', '복', '한', '▁연', '말', '▁되', '세', '요', '.']
[2605, 3776, 236, 3791, 53, 4008, 3780, 3798, 3786, 228, 4035, 3788, 61, 4004, 226, 3853, 3939, 3778]



## 4. Naver movie review 데이터 전처리

In [16]:
import pandas as pd

vocab_file = './kowiki.model'
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

""" train data 준비 """
def prepare_train(vocab, infile, outfile):
    df = pd.read_csv(infile, sep="\t", engine="python")
    with open(outfile, "w") as f:
        for index, row in df.iterrows():
            document = row["document"]
            if type(document) != str:
                continue
            instance = { "id": row["id"], "doc": vocab.encode_as_pieces(document), "label": row["label"] }
            f.write(json.dumps(instance))
            f.write("\n")
            
naver_movie_path = '../../data/naver_movie'

prepare_train(vocab, 
              f'{naver_movie_path}/ratings_train.txt', 
              'naver_movie_ratings_train.json')
prepare_train(vocab, 
              f'{naver_movie_path}/ratings_test.txt', 
              'naver_movie_ratings_test.json')


In [13]:
naver_movie_path = '../../data/naver_movie'
f'{naver_movie_path}/ratings_train.txt'

'../../data/naver_movie/ratings_train.txt'